**Creación base de datos**


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import scipy.integrate as integrate
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from astropy import constants as const
from astropy import units as u

## Defyning constanst 

In [2]:
# Definiendo constantes
G = const.G.to(u.cm**3/(u.g*u.s**2)) # Constante Gravitacional en cm^3/(g*s^2)
Msol = const.M_sun.to(u.g) # Masa solar en gramos
R_sol = const.R_sun.to(u.cm)  # Radio del sol en centímetros

## Unidades

>- Temperature [K]
>- Mass [Earth Mass]
>- Radius [Earth Radius]
>- Distance [AU]
>- Metallicity [Fe/H]

**Open catalogue exoplanets**


>- Calcular flujo de radiación
> . Oblicuidad

In [3]:
# Read the CSV file with space as the delimiter
exoplanet_data = pd.read_csv('exoplanet_data.csv', sep=',', comment='#')
exoplanet_data

Planet_name       Host_name                       gaia_id  \
0            K2-111 b          K2-111    Gaia DR2 53006669599267328   
1          TOI-1695 b        TOI-1695   Gaia DR2 534988616816537728   
2            K2-170 b          K2-170  Gaia DR2 2599451994385832448   
3       Kepler-1038 c     Kepler-1038  Gaia DR2 2079028676836456960   
4       Kepler-1398 b     Kepler-1398  Gaia DR2 2128776801985730048   
..                ...             ...                           ...   
700       Kepler-99 b       Kepler-99  Gaia DR2 2076871091425583232   
701       HIP 29442 d        HD 42813  Gaia DR2 2993561629444856960   
702        TOI-1467 b        TOI-1467   Gaia DR2 403141126645929728   
703  EPIC 201595106 b  EPIC 201595106  Gaia DR2 3701123406596146048   
704       TOI-1798.02        TOI-1798  Gaia DR2 1505890751741673856   

     Number_of_stars  Number_of_planets  Number_of_moons  \
0                  1                  2                0   
1                  1                  1                0   
2                  1                  2                0   
3                  1                  2                0   
4                  1                  2                0   
..               ...                ...              ...   
700                2                  1                0   
701                1                  3                0   
702                1                  1                0   
703                1                  2                0   
704                1                  2                0   

                                    disc_facility  Planet_controversial_flag  \
0                                              K2                          0   
1    Transiting Exoplanet Survey Satellite (TESS)                          0   
2                                              K2                          0   
3                                          Kepler                          0   
4                                          Kepler                          0   
..                                            ...                        ...   
700                                        Kepler                          0   
701                           Paranal Observatory                          0   
702  Transiting Exoplanet Survey Satellite (TESS)                          0   
703                                            K2                          0   
704  Transiting Exoplanet Survey Satellite (TESS)                          0   

     Orbital_period[days]  delta_orbpererr1  ...  sy_vmag  sy_vmagerr1  \
0                5.351800      4.000000e-04  ...   11.130        0.034   
1                3.134279      7.100000e-06  ...   12.989        0.081   
2                7.576544      1.841000e-03  ...   12.668        0.114   
3                7.127660               NaN  ...   15.030        0.229   
4                2.788157      2.021000e-05  ...   13.752        0.103   
..                    ...               ...  ...      ...          ...   
700              4.603580               NaN  ...   13.206        0.092   
701              6.429575      2.600000e-05  ...    9.490        0.030   
702              5.971147      8.746000e-06  ...   12.293        0.017   
703              0.877240      4.000000e-05  ...   11.746        0.069   
704              0.437812      5.300000e-07  ...   11.361        0.023   

     sy_vmagerr2    sy_kmag  sy_kmagerr1  sy_kmagerr2  sy_gaiamag  \
0         -0.034   9.377000        0.021       -0.021    10.92940   
1         -0.081   8.818000        0.021       -0.021    12.13640   
2         -0.114  11.075000        0.021       -0.021    12.49590   
3         -0.229  12.782000        0.022       -0.022    14.85610   
4         -0.103  12.355000        0.021       -0.021    13.66050   
..           ...        ...          ...          ...         ...   
700       -0.092  10.757000        0.018       -0.018    12.96180   
701       -0.030   7.587000       

In [4]:
# Strip whitespace and fix hidden characters
exoplanet_data.columns = exoplanet_data.columns.str.strip()
exoplanet_data.columns = exoplanet_data.columns.str.replace('\u200b', '', regex=False)
exoplanet_data['Planet_name']

0              K2-111 b
1            TOI-1695 b
2              K2-170 b
3         Kepler-1038 c
4         Kepler-1398 b
             ...       
700         Kepler-99 b
701         HIP 29442 d
702          TOI-1467 b
703    EPIC 201595106 b
704         TOI-1798.02
Name: Planet_name, Length: 705, dtype: object

In [ ]:
# 🔧 CONFIGURACIÓN
modo_automatico = False  # Cambia a True para pruebas sin interacción
respuesta_default_cambio = "Y"
respuesta_default_nan = "Y"
valor_default = "42"

# 🧠 Función para manejar input con modo automático
def pedir_input(pregunta, valor_por_defecto=""):
    if modo_automatico:
        print(f"{pregunta} [Auto: {valor_por_defecto}]")
        return valor_por_defecto
    else:
        return input(pregunta)

# 🛰️ Lógica principal
cambios_realizados = 0
guardar_cada = 5
historial = []

for i in exoplanet_data.columns:
    if exoplanet_data[i].isna().any():
        print(f"\n🔍 Revisando columna: {i}")
        a = pedir_input("¿Desea cambiar el valor de la columna? (Y/N): ", respuesta_default_cambio).strip().upper()
        if a == "N":
            continue
        else:
            print(f" Proceda a modificar los valores de la columna '{i}' que son NaN.")

        for idx in exoplanet_data[exoplanet_data[i].isna()].index:
            planeta = exoplanet_data.at[idx, 'Planet_name']
            print(f"\n Planeta: {planeta}, Fila: {idx}, Columna: '{i}'")

            while True:
                r = pedir_input("¿Es diferente a NaN? (Y/N): ", respuesta_default_nan).strip().upper()
                if r in ['Y', 'N']:
                    break
                print(" Entrada no válida. Por favor escribe 'Y' o 'N'.")

            if r == 'Y':
                valor = pedir_input(f"Ingrese un valor para '{i}' en fila {idx} (o escribe 'volver' para deshacer el último cambio): ", valor_default).strip()

                if valor.lower() == "volver":
                    if historial:
                        fila_ant, col_ant, val_ant = historial.pop()
                        exoplanet_data.at[fila_ant, col_ant] = val_ant
                        cambios_realizados -= 1
                        print(f"↩️ Último cambio deshecho en fila {fila_ant}, columna '{col_ant}'.")
                        continue
                    else:
                        print("⚠️ No hay cambios anteriores para deshacer.")
                        continue

                valor_anterior = exoplanet_data.at[idx, i]
                historial.append((idx, i, valor_anterior))

                exoplanet_data.at[idx, i] = valor
                cambios_realizados += 1
                print(f"✅ Valor guardado: {valor}")

            elif r == 'N':
                print("🔁 Se mantiene como NaN.")

            # Guardado automático
            if cambios_realizados % guardar_cada == 0:
                exoplanet_data.to_csv("exoplanet_data.csv", index=False)
                print(f" {cambios_realizados} cambios guardados en el archivo.")

# Guardado final
exoplanet_data.to_csv("exoplanet_data.csv", index=False)
print("\n Guardado final completado.")

✅ Valor guardado: G0

 Planeta: Kepler-914 b, Fila: 97, Columna: 'Spectral_type'
✅ Valor guardado: G1

 Planeta: Kepler-923 b, Fila: 98, Columna: 'Spectral_type'
✅ Valor guardado: G1

 Planeta: OGLE-2019-BLG-1053L b, Fila: 99, Columna: 'Spectral_type'
✅ Valor guardado: M9

 Planeta: EPIC 206317286 b, Fila: 104, Columna: 'Spectral_type'
✅ Valor guardado: M9
 65 cambios guardados en el archivo.

 Planeta: Kepler-1445 b, Fila: 106, Columna: 'Spectral_type'
